In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-20 14:02:30.330897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 14:02:30.896526: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-20 14:02:31.516489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 14:02:31.539701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
subject = 'Goiás - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [5]:
data = pd.read_csv('2003_mo_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Goiás - Consumo de Cimento (t)
0,2003-1,56.459960,0.745293,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,...,18.270239,73.649051,1.213018,0.260097,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,8.179785,97.447
1,2003-2,56.789152,0.745405,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,...,17.704093,73.006649,1.243670,0.258964,9.383590e+07,5.523290e+06,14.034476,8.422082e+07,8.174086,97.256
2,2003-3,57.140263,0.745517,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,...,16.667652,71.378895,1.262206,0.257352,9.395358e+07,5.528254e+06,14.037682,8.425916e+07,8.168386,101.442
3,2003-4,57.649192,0.745629,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,...,15.045930,67.632317,1.355424,0.255929,9.407126e+07,5.533219e+06,14.040888,8.429750e+07,8.162687,98.217
4,2003-5,58.213295,0.745741,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,...,13.454185,63.216002,1.434779,0.254109,9.418894e+07,5.538184e+06,14.044093,8.433584e+07,8.156988,112.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2002-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.317632,55.816672,1.586794,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,2002-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.711263,59.778308,1.450148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,2002-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.095889,64.081346,1.302682,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,2002-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.293938,69.136349,1.253215,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,0.258583,...,0.368966,1.185364,1.275432,-0.492693,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.109783,...,-0.183839,0.918794,1.172452,-0.295685,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.489401,...,-0.942839,0.430785,0.911516,-0.176554,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.617995,...,-1.685236,-0.332805,0.310923,0.422562,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.432536,...,-2.139903,-1.082280,-0.397031,0.932583,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.235206,...,0.278906,-0.826130,-1.027305,-0.903913,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503
188,1.677410,0.682578,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-0.149901,...,0.659967,-0.316346,-0.602530,-1.079101,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711
189,1.732332,0.638983,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-0.186849,...,0.861358,0.274186,-0.068507,-1.160337,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918
190,1.763870,0.595389,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.169383,...,0.883410,0.548970,0.254095,-1.303024,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
247        NaN
248        NaN
249        NaN
250        NaN
251        NaN
Name: Goiás - Consumo de Cimento (t), Length: 252, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,0.258583,...,0.368966,1.185364,1.275432,-0.492693,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.109783,...,-0.183839,0.918794,1.172452,-0.295685,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.489401,...,-0.942839,0.430785,0.911516,-0.176554,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.617995,...,-1.685236,-0.332805,0.310923,0.422562,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.432536,...,-2.139903,-1.082280,-0.397031,0.932583,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.623245,1.561651,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,-0.257374,...,1.381481,0.752880,0.275146,-0.906175,0.944351,1.175698,-1.121085,-0.108013,0.955652,1.757990
158,0.644558,1.540971,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,-0.383476,...,0.653347,0.290362,0.009062,-0.715165,0.952621,1.178649,-1.137848,-0.116498,0.956673,1.696610
159,0.663382,1.520292,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,-0.103699,...,0.008734,-0.452155,-0.599619,-0.488086,0.960982,1.181601,-1.154611,-0.124984,0.957693,1.635230
160,0.670170,1.499612,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,0.154067,...,-0.452584,-0.985366,-0.998607,-0.390703,0.969363,1.184552,-1.171374,-0.133469,0.958714,1.573850


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
157    146.310
158    191.374
159    164.318
160    211.049
161    198.789
Name: Goiás - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [11]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [12]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
126,0.798828,0.605338,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,-0.354087,...,-0.755457,-1.034540,-0.922670,0.288222,0.673675,0.830615,0.695340,0.936426,0.971320,0.568035
127,0.774031,0.608897,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,-0.125102,...,-0.151240,-0.740810,-0.820505,0.195956,0.687412,0.848214,0.652086,0.905979,0.976868,0.664626
128,0.731137,0.612457,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,0.720628,...,0.252355,-0.087802,-0.222596,0.096964,0.706891,0.865813,0.608833,0.875531,0.982416,0.761218
129,0.695886,0.616016,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,0.497599,...,0.305085,0.377084,0.288584,-0.478819,0.722784,0.883412,0.565579,0.845084,0.987963,0.857809
130,0.655420,0.619576,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.525779,...,0.311461,0.643926,0.611816,-0.661613,0.738712,0.901011,0.522326,0.814636,0.993511,0.954400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.235206,...,0.278906,-0.826130,-1.027305,-0.903913,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503
188,1.677410,0.682578,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-0.149901,...,0.659967,-0.316346,-0.602530,-1.079101,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711
189,1.732332,0.638983,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-0.186849,...,0.861358,0.274186,-0.068507,-1.160337,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918
190,1.763870,0.595389,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.169383,...,0.883410,0.548970,0.254095,-1.303024,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126


In [13]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [14]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [15]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_val_loss = min(history.history['val_loss'][500:])
        current_loss = history.history['loss'][history.history['val_loss'].index(current_val_loss)]
        decision_value = (current_val_loss*current_val_loss)*current_loss 
        print(f"val_loss: {current_val_loss}; loss={current_loss} => decision_value={decision_value}")

        if decision_value < best_loss:
            best_loss = decision_value
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[176486487, 3583351475, 4277704663, 3340087248, 1241177127, 3891339860, 4239838054, 1909992898, 4121599601, 1995828354, 1334903064, 4067900579, 3702358768, 3281652312, 3540504387, 3845364678, 3997812295, 81344714, 2570125144, 2171868277]


Step: 0 ___________________________________________


2023-10-20 14:02:31.914451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 14:02:31.914721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 14:02:31.914962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 424.3303527832031; loss=75.22940063476562 => decision_value=13545523.639635779
winner_seed: 176486487


Step: 1 ___________________________________________
val_loss: 451.2257080078125; loss=67.21703338623047 => decision_value=13685699.855376655


Step: 2 ___________________________________________
val_loss: 395.3792724609375; loss=108.422607421875 => decision_value=16949139.069549013


Step: 3 ___________________________________________
val_loss: 496.3170166015625; loss=50.54942321777344 => decision_value=12451868.788845373
winner_seed: 3340087248


Step: 4 ___________________________________________
val_loss: 400.0499267578125; loss=93.23825073242188 => decision_value=14921844.416451273


Step: 5 ___________________________________________
val_loss: 402.5275573730469; loss=88.80725860595703 => decision_value=14389301.079249853


Step: 6 ___________________________________________
val_loss: 402.1927490234375; loss=1066.9503173828125 => decision_value=172588824.24978113


Step

In [18]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 117ms/step - loss: 38844.1445 - val_loss: 1682.8286
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 1931.6427 - val_loss: 1711.5228
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 2350.9927 - val_loss: 1785.1971
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 1881.6298 - val_loss: 1825.5627
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 2451.3635 - val_loss: 1647.3384
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 1819.5186 - val_loss: 1610.2289
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 1752.9612 - val_loss: 1609.0536
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 1697.6838 - val_loss: 2115.0354
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 2113.5686 - val_loss: 1394.0728
Epoch 10/10000
4/4 [==============================] 

4/4 [==============================] - 0s 7ms/step - loss: 676.7586 - val_loss: 1180.1100
Epoch 80/10000
4/4 [==============================] - 0s 7ms/step - loss: 1446.2831 - val_loss: 396.1701
Epoch 81/10000
4/4 [==============================] - 0s 7ms/step - loss: 338.0423 - val_loss: 415.1013
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 276.0804 - val_loss: 579.3776
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 312.7195 - val_loss: 480.0597
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 275.1248 - val_loss: 564.4767
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 315.2519 - val_loss: 365.5742
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 270.2943 - val_loss: 661.8671
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 306.0715 - val_loss: 421.6724
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 230

4/4 [==============================] - 0s 7ms/step - loss: 322.9388 - val_loss: 625.0164
Epoch 158/10000
4/4 [==============================] - 0s 7ms/step - loss: 183.1569 - val_loss: 471.6371
Epoch 159/10000
4/4 [==============================] - 0s 7ms/step - loss: 154.6232 - val_loss: 571.8917
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 149.0936 - val_loss: 573.9741
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 111.7587 - val_loss: 538.3684
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 136.8631 - val_loss: 482.0367
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 122.8534 - val_loss: 497.7690
Epoch 164/10000
4/4 [==============================] - 0s 7ms/step - loss: 190.6308 - val_loss: 601.9178
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 200.0914 - val_loss: 518.1528
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 109.6418 - val_loss: 475.8770
Epoch 236/10000
4/4 [==============================] - 0s 7ms/step - loss: 98.0625 - val_loss: 506.0084
Epoch 237/10000
4/4 [==============================] - 0s 7ms/step - loss: 117.0930 - val_loss: 480.4036
Epoch 238/10000
4/4 [==============================] - 0s 7ms/step - loss: 90.3342 - val_loss: 504.8353
Epoch 239/10000
4/4 [==============================] - 0s 7ms/step - loss: 129.8334 - val_loss: 481.1508
Epoch 240/10000
4/4 [==============================] - 0s 7ms/step - loss: 85.2000 - val_loss: 467.4617
Epoch 241/10000
4/4 [==============================] - 0s 7ms/step - loss: 132.0725 - val_loss: 488.1890
Epoch 242/10000
4/4 [==============================] - 0s 7ms/step - loss: 136.5317 - val_loss: 433.1129
Epoch 243/10000
4/4 [==============================] - 0s 7ms/step - loss: 102.2183 - val_loss: 440.0296
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss:

4/4 [==============================] - 0s 7ms/step - loss: 78.3385 - val_loss: 449.1508
Epoch 314/10000
4/4 [==============================] - 0s 7ms/step - loss: 82.3221 - val_loss: 447.2617
Epoch 315/10000
4/4 [==============================] - 0s 7ms/step - loss: 88.8668 - val_loss: 473.5187
Epoch 316/10000
4/4 [==============================] - 0s 7ms/step - loss: 75.7125 - val_loss: 472.5204
Epoch 317/10000
4/4 [==============================] - 0s 7ms/step - loss: 107.2432 - val_loss: 445.7045
Epoch 318/10000
4/4 [==============================] - 0s 7ms/step - loss: 80.0663 - val_loss: 478.2691
Epoch 319/10000
4/4 [==============================] - 0s 7ms/step - loss: 95.3656 - val_loss: 449.4138
Epoch 320/10000
4/4 [==============================] - 0s 7ms/step - loss: 85.7535 - val_loss: 476.9340
Epoch 321/10000
4/4 [==============================] - 0s 7ms/step - loss: 73.3585 - val_loss: 464.6543
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 69.9

...

4/4 [==============================] - 0s 7ms/step - loss: 21.1170 - val_loss: 488.8520
Epoch 1024/10000
4/4 [==============================] - 0s 7ms/step - loss: 20.9127 - val_loss: 488.8386
Epoch 1025/10000
4/4 [==============================] - 0s 7ms/step - loss: 25.7976 - val_loss: 486.8661
Epoch 1026/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.8794 - val_loss: 479.0695
Epoch 1027/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.4973 - val_loss: 475.5705
Epoch 1028/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.3094 - val_loss: 476.1323
Epoch 1029/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.4792 - val_loss: 515.8483
Epoch 1030/10000
4/4 [==============================] - 0s 7ms/step - loss: 33.7425 - val_loss: 476.8739
Epoch 1031/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.8740 - val_loss: 475.9005
Epoch 1032/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 23.6852 - val_loss: 493.6692
Epoch 1102/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.7150 - val_loss: 492.0333
Epoch 1103/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.8756 - val_loss: 489.8441
Epoch 1104/10000
4/4 [==============================] - 0s 7ms/step - loss: 21.7072 - val_loss: 487.1458
Epoch 1105/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.1095 - val_loss: 492.2489
Epoch 1106/10000
4/4 [==============================] - 0s 7ms/step - loss: 20.0615 - val_loss: 478.7729
Epoch 1107/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.5755 - val_loss: 492.8386
Epoch 1108/10000
4/4 [==============================] - 0s 7ms/step - loss: 24.7201 - val_loss: 473.7273
Epoch 1109/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.3148 - val_loss: 479.6348
Epoch 1110/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 26.2721 - val_loss: 505.2760
Epoch 1180/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.2459 - val_loss: 499.2435
Epoch 1181/10000
4/4 [==============================] - 0s 7ms/step - loss: 21.9241 - val_loss: 500.1513
Epoch 1182/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.6196 - val_loss: 496.0854
Epoch 1183/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.1510 - val_loss: 484.3375
Epoch 1184/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.3950 - val_loss: 486.1373
Epoch 1185/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.2370 - val_loss: 486.6817
Epoch 1186/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.1329 - val_loss: 488.0375
Epoch 1187/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.5887 - val_loss: 488.2917
Epoch 1188/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 20.7028 - val_loss: 502.7147
Epoch 1258/10000
4/4 [==============================] - 0s 7ms/step - loss: 20.3095 - val_loss: 502.5514
Epoch 1259/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.2961 - val_loss: 503.2614
Epoch 1260/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.5536 - val_loss: 502.4731
Epoch 1261/10000
4/4 [==============================] - 0s 7ms/step - loss: 24.5674 - val_loss: 502.3372
Epoch 1262/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.9220 - val_loss: 495.3770
Epoch 1263/10000
4/4 [==============================] - 0s 7ms/step - loss: 33.2721 - val_loss: 493.2345
Epoch 1264/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.3575 - val_loss: 485.3543
Epoch 1265/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.2380 - val_loss: 491.3521
Epoch 1266/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 19.3701 - val_loss: 504.8218
Epoch 1336/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.1959 - val_loss: 507.2616
Epoch 1337/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.7081 - val_loss: 501.5532
Epoch 1338/10000
4/4 [==============================] - 0s 7ms/step - loss: 24.6375 - val_loss: 498.4382
Epoch 1339/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.3127 - val_loss: 501.9914
Epoch 1340/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.1122 - val_loss: 500.5017
Epoch 1341/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.4607 - val_loss: 504.1246
Epoch 1342/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.7961 - val_loss: 502.9989
Epoch 1343/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.3250 - val_loss: 487.6844
Epoch 1344/10000
4/4 [==============================] - 0s 7ms/step - lo

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,199.365555,197.863159,192.132111,179.232727,160.93898,158.950378,158.969147,159.017075,163.917389,173.560516,203.913452,205.141556,190.882782,195.259674,191.985565,186.209259,179.80864,175.502258,169.05365,168.001907,168.123077,174.888397,179.791504,176.398239,177.776184,180.51712,177.465179,174.129028,174.104538,170.106201
Target,205.551,230.558,209.621,210.984,189.889,143.721,176.275,152.632,177.876,181.384,160.12,236.951,206.557,224.27,213.871,211.066,182.899,159.002,178.918,161.936,173.792,176.627,207.991,185.784,234.176,226.996,218.141,234.939,199.619,168.797
Error,6.18544,32.694839,17.488892,31.751266,28.950027,15.229385,17.305847,6.385071,13.958618,7.823486,43.793457,31.809448,15.674225,29.01033,21.885437,24.856735,3.090363,16.500259,9.864349,6.065903,5.66893,1.738602,28.199493,9.385757,56.399811,46.478882,40.675827,60.809967,25.514465,1.309204


In [21]:
display(mae)
display(mape)

21.883478

0.10757462

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined